<a href="https://colab.research.google.com/github/onuralpArsln/MlAiTutorialProjects/blob/main/ImageClass/CatRabbitSqrll/modelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch ile Kendi Datasetinden Eğitim


Bir çok tutorialda torchvision dataset içinden hızlıca bir dataset çekmek tercih edilir çünkü çok kolayca düzgün bir dataset oluşturmuş olursun.

Biz bugün kendi elimizdeki bir data seti kullanarak bu işlemleri yapacağız.

Bunun için bir bulk downloader uzantısı ile googledan bir çok resmi hızlıca kayıt edebilir ya da elimizdeki hazır bir seti kullanabiliriz. Araştırmalar sırasında video çekip bu videodan bir python scripti ile ekran görüntüleri almak gibi teknikler ile de veri setleri hızlıca oluşturulabilir

ancak data bir miktar düzenlemeye ihtiyaç duyabilir.



# Database yapısı


Oldukça basit bir yapıyı takip edeceğiz
                                      
data/                                           
├─ cat/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....             
├─ rabbit/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....      
├─ squirrel/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....         

Bu dosya hem zipli hem de sıkıştırılmamış olarak repoda mevcut. Zipli hali colabda çalışırken wget ile kolayca indirmen için

Colabde isen kolayca indir

In [2]:
!wget https://github.com/onuralpArsln/MlAiTutorialProjects/raw/main/ImageClass/CatRabbitSqrll/data.zip

--2024-07-18 09:30:45--  https://github.com/onuralpArsln/MlAiTutorialProjects/raw/main/ImageClass/CatRabbitSqrll/data.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/onuralpArsln/MlAiTutorialProjects/main/ImageClass/CatRabbitSqrll/data.zip [following]
--2024-07-18 09:30:45--  https://raw.githubusercontent.com/onuralpArsln/MlAiTutorialProjects/main/ImageClass/CatRabbitSqrll/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76061926 (73M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  72.54M  85.2MB/s    in 0.9s    

2024-07-18 09:30:50 (85.2 

In [3]:
!unzip -q data.zip

# Define Transforms

Transfromların olay resimlere rastgele değişimler uygulamaktır.  

Bu hem elimizdeki data miktarını arttırmak için güzel bir metot hem de farklı koşullar altında da tanıma yapabilmek için önemli.

Örneğin rastgele döndürmeler eklemek sayesinde eğer kedi baş aşağı durursa da onu tanıyacağız.





In [4]:
import torchvision.transforms as transforms

In [5]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5)
])

# Data setimizi aktarmak

Bu noktada bizim data setimiz düzenli olduğu için bir metot kullanacağız ancak düzensiz bir data setin var ise nasıl çalışman gerektiğinin örneğide olacak


Ayrıca bizim data setimizde iki sınıf içinde yakın miktarda görsel var. Bu iyi bir dataset için önemli bir yapıdır.

In [6]:
import torchvision
from torchvision.datasets import ImageFolder

In [7]:
dataset = torchvision.datasets.ImageFolder(root='data', transform=transform)

# Custom Dataset

Eğer dataset dosya yapın bizim örnekteki gibi düzenli değil ise  aşağıdaki methodu uygulayacaksın. Ama düzenli bir datasetine sahip olmak her zaman daha iyidir.

__init__(), __len__(), ve __get_item__() methodlarını tanımlamak zorundasın sistemin çalışması için

In [ ]:
import glob
import os
from torch.utils.data import Dataset
from torchvision.io import read_image, ImageReadMode
#glob ve os sistem dizin kontrolü için


class CustomDataset(Dataset):
    # transform ve dosya adresi bilgilerini alarak objeyi başlatır.
    def __init__(self, root_dir, transform):
        self.transform = transform
        self.image_paths = []
        # tercih edilen dosya uzantılarını buraya yaz bizde sadece png ve jpeg mevcut
        for ext in ['png', 'jpg']:
            self.image_paths += glob.glob(os.path.join(root_dir, '*', f'*.{ext}'))
        # set oluşturmak her dizine sadece bir kere uğramak için
        class_set = set()
        # mevcut olan her resim adresini ekle
        for path in self.image_paths:
            class_set.add(os.path.dirname(path))
        self.class_lbl = { cls: i for i, cls in enumerate(sorted(list(class_set)))}
    #  veri  setinin boyutunu döndürebilmek için bir method
    def __len__(self):
        return len(self.image_paths)
    # idx indexideki veriyi yükler
    def __getitem__(self, idx):
        img = read_image(self.image_paths[idx], ImageReadMode.RGB).float()
        cls = os.path.basename(os.path.dirname(self.image_paths[idx]))
        label = self.class_lbl[cls]
        # label bilgisinide paylaşır
        return self.transform(img), torch.tensor(label)

In [ ]:
custom_dataset = CustomDataset('data/', transform)

# Split Data


Amacımız test, validation  ve train oluşturmak.

train-> eğitim için kullanılcak veri

validation -> eğitim sırasında parametre optimizasyonu için kullanılan veri kısmı

test-> model eğitildikten sonra performans ölçümü için kullanılır.

In [8]:
splits = [0.8, 0.1, 0.1]

In [9]:
split_sizes = []
for sp in splits[:-1]:
    split_sizes.append(int(sp * len(dataset)))
split_sizes.append(len(dataset) - sum(split_sizes))

In [10]:
import torch
import torch.utils
train_set, test_set, val_set = torch.utils.data.random_split(dataset, split_sizes)

In [11]:
train_set, test_set, val_set = torch.utils.data.random_split(dataset, split_sizes)

In [12]:
val_set.transform = test_set.transform = transforms.Compose([
    transforms.Resize((128, 128))
])

# DataLoader

Dataloader pytorch içinde iterasyona izin veren bir yapıdır. böylece verilerini pytorch içinde kullanılacak bir yapıya çevirirsin.

In [13]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [14]:
dataloaders = {
    "train": DataLoader(train_set, batch_size=16, shuffle=True),
    "test": DataLoader(test_set, batch_size=16, shuffle=False),
    "val": DataLoader(val_set, batch_size=16, shuffle=False)
}

# Define the Classifier

Sınıflandırmayı yapan modele classifier denir.

her nöral yapı gibi classifier de katmanlardan oluşur. bu katmanları istersek kendimiz elle belirleyebiliriz. Ancak uzmanların hazırladığı güçlü yapılarda pytorch içinden kolayca ulaşılabilir.  Bu tutorialde amacımız öğrenmek olduğu için nasıl yapılıyor göreceğiz ama bir mühendislik uygulaması sırasında iyi ve hazır bir nöral yapıyı kullanmak iyi fikirdir.

## Custom Architecture

aşağıda iki farklı yapı var. eğer ikiden çok çıkışın / sınıfın varsa çok sayıda nöron gerekir

1 0 0      kedi       
0 1 0      tavşan                        
0 0 1      sincap                    
gibi            

ancak eğer iki  çıkış lazımsa            
1 kuş             
0 tavşan                   

yapabilirsin.


In [ ]:
## Çok sınıflı sınıflandırma için


class CustomModel(torch.nn.Module):   ## custom model yapmak için miras alınıyor
    def __init__(self):
        super(CustomModel, self).__init__() ## miras alınan parentin init çalışıyor

        # linear ( fully connected) bir giriş 128 boyutlu girdi alıyor ve
        #256 boyutunda bir tensor dışarı veriyor
        self.linear1 = torch.nn.Linear(128, 256)

        # ReLu Rectified Linear Unit aktivasyonu ilk layerdan sonra kullanılıyor
        self.activation = torch.nn.ReLU()

        # aşağıdaki 3 sayısı 3 sınıf olduğu için 4 olsa 4 yazardık
        # iki katman var giriş ve çıkış 128 girdi 3 çıktı
        self.linear2 = torch.nn.Linear(256, 3)

        # soft max katmanı çıktıyı olasılığa dönüştürür.
        self.softmax = torch.nn.Softmax()

  # ileri yönde modelde harekti yapıyoruz
  # x datasını katmanlardan geçiriyor ve sonucu döndürüyor
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

In [ ]:
## Binary ikili sınıflandırma içi
class CustomModel(torch.nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()

        self.linear1 = torch.nn.Linear(128, 256)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(256, 1)
        self.softmax = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x